In [5]:
import os
import copy
import numpy as np
import pandas as pd
from PIL import Image

class ImageConcat:
    def __init__(self, n_h, n_v, resolution=1000):
        self.n_h = n_h
        self.n_v = n_v
        self.r = resolution
        self.img = Image.new("L", (self.n_h*self.r, self.n_v*self.r), color=255)# color=(255,255,255))
        self.cursor = [0,0]
        self.max_h = 0
    
    def move_cursor_h(self, r):
        self.cursor[0] += r
        if self.cursor[0] > self.max_h:
            self.max_h = copy.deepcopy(self.cursor[0])
        
    def move_cursor_v(self):
        self.cursor[1] += self.r
        
    def reset_cursor_h(self):
        self.cursor[0] = 0
        
    def append_img(self, path, cut_margin=True, clip_white=False):
        try:
            img = Image.open(path).convert('L').resize((self.r,self.r), Image.BILINEAR)
            if cut_margin:
                img = self.cut_margin(img, clip_white=clip_white)
            self.img.paste(img, tuple(self.cursor))
            return img.size
        except:
            # print(f"Not Appended: {path}")
            return None
        
    def add_letter(self, path, cut_margin=True, clip_white=False):
        imgsize = self.append_img(path, cut_margin=cut_margin, clip_white=clip_white)
        if imgsize is None:
            self.move_cursor_h(self.r)
        else:
            self.move_cursor_h(imgsize[0])
    
    def add_space(self):
        self.move_cursor_h(int(self.r/3))
        
    def enter(self):
        self.move_cursor_v()
        self.reset_cursor_h()
        
    def cut_margin(self, img, clip_white):
        npimg = 255 - np.array(img)
        if clip_white:
            npimg[np.where(npimg<10)] = 0 
        wmin = npimg.sum(0).nonzero()[0].min()
        wmax = npimg.sum(0).nonzero()[0].max()
        # hmin = npimg.sum(1).nonzero()[0].min()
        # hmax = npimg.sum(1).nonzero()[0].max()

        npimg = 255 - npimg[:,wmin:wmax+1]
        img = Image.fromarray(npimg)
        return img

# Semantic Segmentation 학습데이터 예시

In [20]:
import os
import random

fd = "/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled"

fonts = os.listdir(fd)

letters = '가 깨 냐 댸 떠 레 며 볘 삐 사 쌔 야 쟤 쩌 체 켜 톄 피 하 교 꾸 뉴 드 또 료 무 뷰 쁘 소 쑈 우 쥬 쯔 초 쿄 투 퓨 흐 귀 끠 놔 돼 뙤 뤄 뭬 뷔 쁴 솨 쐐 외 줘 쮀 취 킈 톼 퐤 회 걕 꺾 넧 뎐 뗹 릲 몯 뵬 뿕 슒 쓻 왌 좵 쬞 춿 퀨 튑 픲 핫 갰 꺙 넂 덫 뗔 렽 몦 빟'
letters = letters.split(" ")

In [25]:
img = ImageConcat(n_h=8, n_v=len(letters), resolution=96)
for l in letters:
    for _ in range(8):
        font = random.choice(fonts)
        img.add_letter(path=f"{fd}/{font}/{font}__{l}.png", cut_margin=False)
    img.enter()

In [27]:
img.img.save("semantic_segmentation_train_example.png")

# Semantic Segmentation 인퍼런스 예시

In [28]:
fd = "/home/jupyter/ai_font/exp0717/segmentation"

In [36]:
files = [f for f in os.listdir(fd) if f.endswith("1_orig.png")]

In [37]:
f = files[0]

In [39]:
f.split("_")

['test', '57', '18', '1', 'orig.png']

In [43]:
img = ImageConcat(n_h=4, n_v=64, resolution=96)
for f in files[:64]:
    fs = f.split("_")
    img.add_letter(path=f"{fd}/{f}", cut_margin=False)
    img.add_letter(path=f"{fd}/test_{fs[1]}_{fs[2]}_1.png", cut_margin=False)
    img.add_letter(path=f"{fd}/test_{fs[1]}_{fs[2]}_2.png", cut_margin=False)
    img.add_letter(path=f"{fd}/test_{fs[1]}_{fs[2]}_3.png", cut_margin=False)
    img.enter()

In [44]:
img.img.save("semantic_segmentation_test_example.png")

# Custom Algorithm 예시

In [46]:
hk = "가까나다따라마바빠사싸아자짜차카타파하"
vk = "구꾸누두뚜루무부뿌수쑤우주쭈추쿠투푸후"
k = "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"
fd = "/home/jupyter/ai_font/data/exp0717/train0730_whole"
hfd = "/home/jupyter/ai_font/data/exp0717/train0730_hparts"
vfd = "/home/jupyter/ai_font/data/exp0717/train0730_vparts"
fonts = [f for f in sorted(os.listdir(fd)) if not os.path.isdir(f)]

In [48]:
img = ImageConcat(n_h=len(k), n_v=16*4, resolution=96)
for font in fonts[:16]:
    for l in hk:
        img.add_letter(path=f"{fd}/{font}/{font}__{l}.png", cut_margin=False)
    img.enter()
    for l in k:
        img.add_letter(path=f"{hfd}/{font}/{font}__{l}.png", cut_margin=False)
    img.enter()
    for l in vk:
        img.add_letter(path=f"{fd}/{font}/{font}__{l}.png", cut_margin=False)
    img.enter()
    for l in k:
        img.add_letter(path=f"{vfd}/{font}/{font}__{l}.png", cut_margin=False)
    img.enter()

In [50]:
img.img.save("custom_algo_example.png")

# Download and Unzip Report files

In [52]:
import os
import zipfile
from google.cloud import storage

def download_file_from_bucket(bucket_name, source_blob_name, destination_file_name):
    """
    Downloads a file from the Google Cloud Storage bucket.
    
    :param bucket_name: Name of the GCP bucket.
    :param source_blob_name: Name of the blob in the bucket.
    :param destination_file_name: Path to save the downloaded file.
    """
    # Create a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob (file in GCP bucket)
    blob = bucket.blob(source_blob_name)

    # Download the file
    blob.download_to_filename(destination_file_name)
    print(f"File {source_blob_name} downloaded to {destination_file_name}.")

def unzip_file(zip_file_path, extract_dir):
    """
    Unzips a ZIP file into the specified directory.
    
    :param zip_file_path: Path to the ZIP file.
    :param extract_dir: Directory to extract the contents to.
    """
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"File {zip_file_path} unzipped to {extract_dir}.")

    
bucket_name = "leo_font"
source_blob_name = "exp0717/report/report0730_plain.zip"  # Path in the bucket
destination_file_name = "report0730_plain.zip"
extract_dir = "/home/jupyter/ai_font/data/exp0717/report0730_plain"

# Step 1: Download the ZIP file from the GCP bucket
download_file_from_bucket(bucket_name, source_blob_name, destination_file_name)

# Step 2: Unzip the downloaded file
unzip_file(destination_file_name, extract_dir)


File exp0717/report/report0730_plain.zip downloaded to report0730_plain.zip.
File report0730_plain.zip unzipped to /home/jupyter/ai_font/data/exp0717/report0730_plain.


In [56]:
name = "test0730_handcut_whole"

bucket_name = "leo_font"
source_blob_name = f"exp0717/report/{name}.zip" # Path in the bucket
destination_file_name = f"{name}.zip"
extract_dir = f"/home/jupyter/ai_font/data/exp0717/{name}"

# Step 1: Download the ZIP file from the GCP bucket
download_file_from_bucket(bucket_name, source_blob_name, destination_file_name)

# Step 2: Unzip the downloaded file
unzip_file(destination_file_name, extract_dir)

File exp0717/report/test0730_handcut_whole.zip downloaded to test0730_handcut_whole.zip.
File test0730_handcut_whole.zip unzipped to /home/jupyter/ai_font/data/exp0717/test0730_handcut_whole.


# 학습결과 시각화

In [81]:
fd = "/home/jupyter/ai_font/data/exp0717/report0730_plain/i0"
files = [f for f in sorted(os.listdir(fd)) if f.endswith(".png")]
font = "플레이브밤비"
realfd = "/home/jupyter/ai_font/data/exp0717/test0730_handcut_whole"

img = ImageConcat(n_h=9, n_v=len(files)//4, resolution=96)
for i in range(0,len(files),4):
    letter = files[i][-5]
    img.add_letter(path=f"{realfd}/{font}/{font}__{letter}.png", cut_margin=False)
    letter = files[i+1][-5]
    img.add_letter(path=f"{realfd}/{font}/{font}__{letter}.png", cut_margin=False)
    letter = files[i+2][-5]
    img.add_letter(path=f"{realfd}/{font}/{font}__{letter}.png", cut_margin=False)
    letter = files[i+3][-5]
    img.add_letter(path=f"{realfd}/{font}/{font}__{letter}.png", cut_margin=False)
    
    img.add_space()
    
    img.add_letter(path=f"{fd}/{files[i]}", cut_margin=False)
    img.add_letter(path=f"{fd}/{files[i+1]}", cut_margin=False)
    img.add_letter(path=f"{fd}/{files[i+2]}", cut_margin=False)
    img.add_letter(path=f"{fd}/{files[i+3]}", cut_margin=False)
    
    img.enter()

In [83]:
img.img.save("result_visualize.png")

In [80]:
len(files)//4

19

# Inpainting 예시

In [107]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import cv2

def get_unpadded_borders(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
    processed = cv2.morphologyEx(np.array(img), cv2.MORPH_CLOSE, kernel)
    npimg = 255 - np.array(processed)
    npimg[np.where(npimg < 16)] = 0
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()
    return wmin, wmax, hmin, hmax

def unpad(img):
    wmin, wmax, hmin, hmax = get_unpadded_borders(img)
    wmin = wmin - 1 if wmin > 0 else wmin
    wmax = wmax + 1 if wmax < img.width else wmax
    hmin = hmin - 1 if hmin > 0 else hmin
    hmax = hmax + 1 if hmax < img.height else hmax
    return Image.fromarray(np.array(img)[hmin:hmax,wmin:wmax])

def resize_aspectratio(image, new_size=128):


    # Get the original dimensions
    original_width, original_height = image.size

    # Set the desired width or height
    desired_width = new_size # Example width; adjust as needed
    desired_height = new_size  # Example height; adjust as needed

    # Calculate the aspect ratio
    aspect_ratio = original_width / original_height

    # Calculate new dimensions while maintaining aspect ratio
    if original_width > original_height:
        new_width = desired_width
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = desired_height
        new_width = int(new_height * aspect_ratio)

    # Resize the image while maintaining aspect ratio
    resized_image = image.resize((new_width, new_height), Image.BILINEAR)
    return resized_image

def centering(img, new_size=128, padding_color=255):

    # Resize the image using the new dimensions
    resized_img = unpad(img)
    resized_img = resize_aspectratio(resized_img)
    
    # Create a new blank image with padding
    padded_img = Image.new("L", (new_size, new_size), padding_color)
    
    # Calculate the position to paste the resized image
    left = (padded_img.width - resized_img.width) // 2
    top = (padded_img.height - resized_img.height) // 2
    right = left + resized_img.width
    bottom = top + resized_img.height
    
    # Paste the resized image onto the blank image
    padded_img.paste(resized_img, (left, top, right, bottom))
    
    return padded_img

In [108]:
img = Image.open("플레이브밤비__갊.png")

In [110]:
centering(img).save("플레이브밤비__갊_커짐.png")

In [119]:
wmin, wmax, hmin, hmax = get_unpadded_borders(img)

In [124]:
npimg = np.ones_like(np.array(img))*int(255)

In [125]:
npimg[hmin:hmax,wmin:wmax] = 230

In [126]:
Image.fromarray(npimg).save("플레이브밤비__갊_마스크.png")

In [127]:
npwhere = np.where(np.array(img)!=255)
npimg[npwhere] = np.array(img)[npwhere]
Image.fromarray(npimg).save("플레이브밤비__갊_마스크생성.png")

In [129]:
img = ImageConcat(n_h=4, n_v=1, resolution=96)
img.add_letter(path="플레이브밤비__갊_커짐.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊_마스크.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊_마스크생성.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊.png", cut_margin=False)


In [131]:
img.img.save("인페인팅리사이즈예시.png")

## 인페인팅 초성/종성 Given

In [147]:
img = Image.open("플레이브밤비__갊_noch23.png")
npimg = np.ones_like(np.array(img))*int(230)
npwhere = np.where(np.array(img)!=255)
# npimg[npwhere] = np.array(img)[npwhere]
npimg[npwhere] = 255
Image.fromarray(npimg).save("플레이브밤비__갊_noch23_shaded.png")

In [148]:
img = Image.open("플레이브밤비__갊_noch2.png")
npimg = np.ones_like(np.array(img))*int(230)
npwhere = np.where(np.array(img)!=255)
# npimg[npwhere] = np.array(img)[npwhere]
npimg[npwhere] = 255
Image.fromarray(npimg).save("플레이브밤비__갊_noch2_shaded.png")

In [149]:
img = ImageConcat(n_h=3, n_v=2, resolution=96)
img.add_letter(path="플레이브밤비__갊_noch23.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊_noch23_shaded.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊.png", cut_margin=False)

img.enter()

img.add_letter(path="플레이브밤비__갊_noch2.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊_noch2_shaded.png", cut_margin=False)
img.add_letter(path="플레이브밤비__갊.png", cut_margin=False)


In [150]:
img.img.save("인페인팅기븐.png")